# Ships demo

Load and visualize vessel locations from July to September 2019

In [ ]:
import time

from datashader.utils import lnglat_to_meters
import holoviews as hv
import holoviews.operation.datashader as hd
from holoviews.element import tiles
from matplotlib import cm
import numpy as np
import pandas as pd
import tiledb

hv.extension("bokeh", "matplotlib")

tiledb_uri = 'tiledb://exactearth/ee_ais'

config = tiledb.Config()

# Set value
config["sm.memory_budget"] = 50_000_000
config["sm.memory_budget_var"] = 50_000_000

ctx = tiledb.Ctx(config)

Query for the area around Southampton, England. Note if querying for large areas then you should consider using a larger notebook or breaking the area into smaller queries and using [user defined functions](https://docs.tiledb.com/cloud/client-api/serverless-array-udfs)

In [ ]:
%%time

t1 = np.datetime64('2019-07-01T00:00:00')
t2 = np.datetime64('2019-07-02T00:00:00')
x1, x2, y1, y2 = [-1.7, -1.2,  50, 51.0]

with tiledb.open(tiledb_uri, ctx=ctx) as arr:
    pts = arr.query(attrs=["mmsi"], dims=["longitude", "latitude"]).multi_index[t1:t2, x1:x2, y1:y2]
    
print(f"Retrieved {len(pts['longitude'])} vessels")

Turn the result into a Pandas dataframe

In [ ]:
df = pd.DataFrame(pts)
df.loc[:, 'x'], df.loc[:, 'y'] = lnglat_to_meters(df.longitude,df.latitude)
df

And visualize using datashader.

In [ ]:
%%time

hv.output(backend="bokeh")
bkgrd = tiles.EsriImagery().opts(xaxis=None, yaxis=None, width=700, height=500)
points = hv.Points(df, ['x', 'y'])
bkgrd * hd.datashade(points, cmap=cm.plasma)